In [3]:
# Instalacja PyTorch Lightning
!pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 60.4 MB/s eta 0:00:00


In [4]:
# 1. System i Dane
import os
import pandas as pd
import numpy as np

# 2. PyTorch (Silnik)
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# 3. PyTorch Lightning (Trener)
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

# 4. Preprocessing
# To jedyny element z zewnątrz, który zostawiamy, bo ręczne pisanie
# normalizacji danych (skalowania do 0-1) to strata czasu i ryzyko błędu.
from sklearn.preprocessing import MinMaxScaler

# Ustawienie ziarna losowości dla powtarzalności
pl.seed_everything(42)

INFO:lightning_fabric.utilities.seed:Seed set to 42


42

In [5]:
!git clone https://github.com/edwardzjl/CMAPSSData.git

Cloning into 'CMAPSSData'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16 (from 1)
Receiving objects: 100% (16/16), 11.96 MiB | 15.44 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [6]:
class CMAPSS_Preprocessor:
    def __init__(self, data_path, sequence_length=30, alpha=0.25, max_rul=125):
        self.data_path = data_path #ściezka do pliku
        self.sequence_length = sequence_length #długość okna czasowego
        self.alpha = alpha  # Współczynnik wygładzania z artykułu
        self.max_rul = max_rul # Przycinanie RUL (Piecewise Linear)

        # Definicja kolumn tablicy na dane
        self.index_cols = ['unit_nr', 'time_cycles']
        self.setting_cols = ['os_1', 'os_2', 'os_3']
        self.sensor_cols = ['s' + str(i) for i in range(1, 22)]
        self.cols = self.index_cols + self.setting_cols + self.sensor_cols

        # Skaler (będziemy go uczyć tylko na treningu)(?) - normalizacja odczytów z czujników do zakresu 0-1
        self.scaler = MinMaxScaler(feature_range=(0, 1))

    def process(self, file_name='FD001'):
        # 1. Wczytanie danych z  plików do dataframes (tabeli)
        train_df = pd.read_csv(f'{self.data_path}/train_{file_name}.txt', sep=r'\s+', header=None, names=self.cols)
        test_df = pd.read_csv(f'{self.data_path}/test_{file_name}.txt', sep=r'\s+', header=None, names=self.cols)
        test_rul_df = pd.read_csv(f'{self.data_path}/RUL_{file_name}.txt', sep=r'\s+', header=None, names=['RUL'])

        # 2. Obliczenie etykiet RUL dla treningu
        train_df = self._add_rul(train_df, is_test=False)

        # 3. Wygładzanie danych (Exponential Smoothing) - WAŻNE!
        train_df = self._smooth_data(train_df)
        test_df = self._smooth_data(test_df)

        # 4. Normalizacja (Fit na treningu, Transform na obu)
        # Bierzemy wszystkie sensory i ustawienia
        feats = self.setting_cols + self.sensor_cols #podpisy kolumn    feats = ['os_1', 'os_2', 'os_3', 's1', 's2', ..., 's21']
        self.scaler.fit(train_df[feats])      # .fit znajduje max i min w zbiorach feats
        train_df[feats] = self.scaler.transform(train_df[feats])  # normalizacja dataframeu treningowego
        test_df[feats] = self.scaler.transform(test_df[feats])  # normalizacja dataframeu testowego

        # 5. Generowanie okien czasowych (Sliding Window)
        X_train, y_train = self._gen_sequence(train_df, feats)
        # Uwaga: Dla testu w C-MAPSS zazwyczaj bierze się tylko OSTATNIE okno,
        # bo plik RUL_FD001.txt zawiera tylko jedną liczbę dla każdego silnika (RUL na samym końcu).
        X_test, y_test = self._gen_test_sequence(test_df, test_rul_df, feats)

        return X_train, y_train, X_test, y_test

    def _add_rul(self, df, is_test=False):
        # Grupowanie po silniku, znalezienie max cyklu
        max_life = df.groupby('unit_nr')['time_cycles'].transform('max')
        df['RUL'] = max_life - df['time_cycles']

        # Implementacja Piecewise Linear RUL (ucinamy powyżej 125)
        df['RUL'] = df['RUL'].clip(upper=self.max_rul)
        return df

    def _smooth_data(self, df):
        # WAŻNE: Musimy grupować po 'unit_nr'!
        # Nie możemy pozwolić, by wygładzanie przeniosło się z końca Silnika 1 na początek Silnika 2.

        df[self.sensor_cols] = df.groupby('unit_nr')[self.sensor_cols].transform(     #wg Gemini .ewm.mean zrobi to samo co wzór na wygładzanie z artykułu
            lambda x: x.ewm(alpha=self.alpha, adjust=False).mean()
        )
        return df

    def _gen_sequence(self, df, feature_cols):
        X, y = [], []
        data_array = df[feature_cols].values    # data_array: Czysta macierz liczb z sensorów i parametrów Wymiar: [20631, 24] -> [Liczba wszystkich cykli, Liczba sensorów]
        target_array = df['RUL'].values     # target_array: Wektor RUL. Wymiar: [20631] -> [Liczba wszystkich cykli]
        unit_ids = df['unit_nr'].values

        # Iterujemy, ale musimy uważać, żeby okno nie "przeskoczyło" między silnikami
        for i in range(len(df) - self.sequence_length):
            # Sprawdzamy czy okno mieści się w JEDNYM silniku
            if unit_ids[i] == unit_ids[i + self.sequence_length]: #id silnika takie same na początku i końcu sekwencji
                # Dodajemy okno (30 wierszy)
                X.append(data_array[i : i + self.sequence_length]) #
                # Dodajemy cel (RUL w ostatnim kroku tego okna)
                y.append(target_array[i + self.sequence_length - 1])

        # np.array(X) ma wymiar: [N, T, K]
        # N (Samples) ≈ 17 631 (Wszystkie cykle minus 30 dla każdego ze 100 silników)
        # T (Time) = 30 (Długość naszego okna)
        # K (Features) = 24 (Liczba sensorów)
        # Kształt: (17631, 30, 24)

        # np.array(y) ma wymiar: [N]
        # Kształt: (17631,) -> Wektor zawierający jedną liczbę RUL dla każdego okna
        return np.array(X), np.array(y)

    def _gen_test_sequence(self, test_df, truth_df, feature_cols):
        X, y = [], []
        # Dla zbioru testowego bierzemy tylko OSTATNIE 30 cykli każdego silnika
        # I przypisujemy mu prawdziwy RUL z pliku RUL_FD001.txt

        true_ruls = truth_df['RUL'].values   #truth_df  Wymiar: [100 wierszy, 1 kolumna] (bo mamy 100 silników testowych i po jednej liczbie RUL dla każdego).

        for unit_id in test_df['unit_nr'].unique():     #.unique usuwa wielokrotne id
            # Wyciągamy dane jednego silnika
            temp_df = test_df[test_df['unit_nr'] == unit_id]

            if len(temp_df) >= self.sequence_length:
                # Bierzemy ostatnie 30 cykli
                window = temp_df[feature_cols].values[-self.sequence_length:]
                X.append(window)
                # Bierzemy prawdziwy RUL (indeks unit_id-1, bo silniki są od 1, tablica od 0)
                y.append(true_ruls[unit_id - 1])

        return np.array(X), np.array(y)

In [7]:
class CMAPSSDataModule(pl.LightningDataModule):
    def __init__(self, data_path='CMAPSSData', batch_size=10, sequence_length=30):
        super().__init__()
        self.data_path = data_path
        self.batch_size = batch_size # Wg artykułu batch=10
        self.sequence_length = sequence_length
        self.preprocessor = CMAPSS_Preprocessor(data_path, sequence_length)

    def setup(self, stage=None):
        # Tu dzieje się cała magia przygotowania danych
        X_train_np, y_train_np, X_test_np, y_test_np = self.preprocessor.process('FD001')

        # Konwersja Numpy -> PyTorch Tensor
        self.train_X = torch.tensor(X_train_np, dtype=torch.float32)
        self.train_y = torch.tensor(y_train_np, dtype=torch.float32).unsqueeze(1) # [N] -> [N, 1]

        self.test_X = torch.tensor(X_test_np, dtype=torch.float32)
        self.test_y = torch.tensor(y_test_np, dtype=torch.float32).unsqueeze(1)

        print(f"Dane przygotowane!")
        print(f"Trening: {self.train_X.shape}") # Oczekiwane: [17631, 30, 24]
        print(f"Test: {self.test_X.shape}")     # Oczekiwane: [100, 30, 24]

    def train_dataloader(self):
        # Shuffle=True jest kluczowe dla treningu!
        dataset = torch.utils.data.TensorDataset(self.train_X, self.train_y)
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        # Używamy zbioru testowego jako walidacji, żeby śledzić postęp (w prawdziwym projekcie można by wydzielić osobny val)
        dataset = torch.utils.data.TensorDataset(self.test_X, self.test_y)
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=False)

In [8]:
# 1. Inicjalizacja DataModule
dm = CMAPSSDataModule(batch_size=10, sequence_length=30)

# 2. Uruchomienie setup (wczytanie i przetworzenie)
dm.setup()

# 3. Sprawdzenie jednej paczki danych
# Pobieramy jeden batch z loadera
x_batch, y_batch = next(iter(dm.train_dataloader()))

print("\n--- Sprawdzenie Batcha ---")
print(f"Wymiary wejścia (X): {x_batch.shape}")
# Powinno być: torch.Size([10, 30, 24]) -> 10 próbek, 30 cykli, 24 cechy
print(f"Wymiary etykiet (y): {y_batch.shape}")
# Powinno być: torch.Size([10, 1]) -> 10 wyników RUL

print("\nPrzykładowy RUL z batcha:", y_batch[0].item())

Dane przygotowane!
Trening: torch.Size([17631, 30, 24])
Test: torch.Size([100, 30, 24])

--- Sprawdzenie Batcha ---
Wymiary wejścia (X): torch.Size([10, 30, 24])
Wymiary etykiet (y): torch.Size([10, 1])

Przykładowy RUL z batcha: 125.0
